# Building a WebScaper to gather Data of Books Selling on Amazon.in

#### For Educational Purposes only

In [2]:
#importing beutiful soup 4 and selenium
from bs4 import BeautifulSoup
import selenium

In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


### Start the webdriver

In [25]:
# startup the webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\boraw\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
<ipython-input-25-a8f3e5a54fdc>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
url = "https://www.amazon.in/"
driver.get(url)

In [274]:
def get_url(search_term):
    """Generates a URL from the search term."""
    generic_url = 'https://www.amazon.in/s?k={}'
    search_term = search_term.replace(' ','+') # conforming to the amazon's url format for search terms with scapes
   
    url = generic_url.format(search_term)
    
    # added page query
    url += '&page={}'
    
    return url

In [252]:
url = get_url('Books')
url

'https://www.amazon.in/s?k=Books&page{}'

In [253]:
driver.get(url)

### Extract the Collection

In [129]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [61]:
results = soup.find_all(class_= "a-section a-spacing-small s-padding-left-small s-padding-right-small")

In [159]:
results = soup.find_all('div',{'data-component-type':'s-search-result'})

In [161]:
len(results)

22

In [176]:
results

[<div class="s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 AdHolder sg-col s-widget-spacing-small sg-col-12-of-16" data-asin="8194899133" data-cel-widget="search_result_1" data-component-id="32" data-component-type="s-search-result" data-index="1" data-uuid="cf88e4ef-c206-4dd8-8245-e69b869fa754"><div class="sg-col-inner">
 <div cel_widget_id="MAIN-SEARCH_RESULTS-1" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-cel-widget="MAIN-SEARCH_RESULTS-1" data-csa-c-id="8r6jqk-vc5htz-rlh0k7-jw4wr">
 <div class="rush-component" data-component-id="33" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://unagi-eu.amazon.com/1/events/com.amazon.eel.SponsoredProductsEventTracking.prod?qualifier=1649696266&amp;id=6708276933000838&amp;widgetName=sp_atf&amp;adId=20025758065698&amp;eventType=1&amp;adIndex=0"}' data-compone

In [224]:
item = results[0]

In [225]:
atag = item.h2.a

In [226]:
title = atag.text.strip()
title

'How To Win Friends & Influence People, International Bestseller'

In [227]:
product_url = "https:amazon.in" + atag.get('href')
product_url

'https:amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A03269371LC14WGRP6KNN&url=%2FFriends-Influence-People-International-Bestseller%2Fdp%2F8194899133%2Fref%3Dsr_1_1_sspa%3Fkeywords%3DBooks%26qid%3D1649696266%26sr%3D8-1-spons%26psc%3D1&qualifier=1649696266&id=6708276933000838&widgetName=sp_atf'

In [228]:
price_parent = item.find('span','a-price')

In [267]:
price_parent.find('span','a-offscreen').text[1:]

'109'

In [230]:
rating = item.i.text
rating

'4.5 out of 5 stars'

In [231]:
rating_count  = item.find('span',{"class":"a-size-base s-underline-text"}).text
rating_count

'63,852'

In [233]:
author = item.find('a',{'class':'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style'}).text.strip()
author

'Dale Carnegie'

### General Function for getting details

In [237]:
def extract_details(item):
    """Extracts details of a single record"""
    atag = item.h2.a 
    title = atag.text.strip()
    
    product_url = "https://amazon.in" + atag.get('href')
    price_parent = item.find('span','a-price')
    price = price_parent.find('span','a-offscreen').text
    rating = item.i.text
    rating_count  = item.find('span',{"class":"a-size-base s-underline-text"}).text
    
    try: # used try block as some author names are missing
        author = item.find('a',{'class':'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style'}).text.strip()
    except AttributeError:
        author = 'unknown'
        
    output = (title,author,product_url,price,rating, rating_count)
    
    return output


In [238]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})

for item in results:
    records.append(extract_details(item))

In [239]:
records

[('How To Win Friends & Influence People, International Bestseller',
  'Dale Carnegie',
  'https://amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A03269371LC14WGRP6KNN&url=%2FFriends-Influence-People-International-Bestseller%2Fdp%2F8194899133%2Fref%3Dsr_1_1_sspa%3Fkeywords%3DBooks%26qid%3D1649696266%26sr%3D8-1-spons%26psc%3D1&qualifier=1649696266&id=6708276933000838&widgetName=sp_atf',
  '₹109',
  '4.5 out of 5 stars',
  '63,852'),
 ('Great Truths That Set Us Free',
  'unknown',
  'https://amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A075246620I5BXDEL7L0X&url=%2FGreat-Truths-That-Set-Free%2Fdp%2F9389053803%2Fref%3Dsr_1_2_sspa%3Fkeywords%3DBooks%26qid%3D1649696266%26sr%3D8-2-spons%26psc%3D1&qualifier=1649696266&id=6708276933000838&widgetName=sp_atf',
  '₹109',
  '4.3 out of 5 stars',
  '121'),
 ('Oswaal CBSE Term 2 Science Class 10 Sample Question Paper Book (For Term-2 2022 Exam)',
  'Oswaal Editorial Board',

In [241]:
import pandas as pd

In [243]:
df = pd.DataFrame(records)

In [250]:
df.head()

,0,1,2,3,4,5
0,"How To Win Friends & Influence People, Interna...",Dale Carnegie,https://amazon.in/gp/slredirect/picassoRedirec...,₹109,4.5 out of 5 stars,"63,852"
1,Great Truths That Set Us Free,unknown,https://amazon.in/gp/slredirect/picassoRedirec...,₹109,4.3 out of 5 stars,121
2,Oswaal CBSE Term 2 Science Class 10 Sample Que...,Oswaal Editorial Board,https://amazon.in/Oswaal-Science-Sample-Questi...,₹111,4.5 out of 5 stars,63
3,The Psychology of Money,Morgan Housel,https://amazon.in/Psychology-Money-Morgan-Hous...,₹321,4.6 out of 5 stars,"30,562"
4,The Power of Your Subconscious Mind,Joseph Murphy,https://amazon.in/Power-Your-Subconscious-Mind...,₹105,4.5 out of 5 stars,"51,029"


In [249]:
df.iloc[0,:]

0    How To Win Friends & Influence People, Interna...
1                                        Dale Carnegie
2    https://amazon.in/gp/slredirect/picassoRedirec...
3                                                 ₹109
4                                   4.5 out of 5 stars
5                                               63,852
Name: 0, dtype: object


# Put it all together

In [281]:
def get_url(search_term):
    """Generates a URL from the search term."""
    generic_url = 'https://www.amazon.in/s?k={}'
    search_term = search_term.replace(' ','+') # conforming to the amazon's url format for search terms with scapes
   
    url = generic_url.format(search_term)
    
    # added page query
    url += '&page={}'
    
    return url
def extract_details(item):
    """Extracts details of a single record"""
    atag = item.h2.a 
    title = atag.text.strip()
    
    product_url = "https://amazon.in" + atag.get('href')
    price_parent = item.find('span','a-price')
    
    
    
    
    try: # used try block for errors due to missing values
        price = price_parent.find('span','a-offscreen').text[1:]
        author = item.find('a',{'class':'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style'}).text.strip()
        rating = item.i.text
        rating_count  = item.find('span',{"class":"a-size-base s-underline-text"}).text
    except AttributeError:
        price = 'unkown'
        author = 'unknown'
        rating = 'unknown'
        rating_count = 'unknown'
       
        
    output = (title,author,product_url,price,rating, rating_count)
    
    return output




def main(search_term):
    """Run Main program"""
    # startup the webdriver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract_details(item)
            if record:
                records.append(record)
    
    driver.close()
    df = pd.DataFrame(records)
    df.columns = ['Title','Author','Link','Price',"Rating",'Rating Count']
    df.to_csv('amazon_books_records.csv', sep=',')


In [291]:
url = get_url('Books')
url

'https://www.amazon.in/s?k=Books&page={}'

In [292]:
for page in range(1,21):
    print(url.format(page))

https://www.amazon.in/s?k=Books&page=1
https://www.amazon.in/s?k=Books&page=2
https://www.amazon.in/s?k=Books&page=3
https://www.amazon.in/s?k=Books&page=4
https://www.amazon.in/s?k=Books&page=5
https://www.amazon.in/s?k=Books&page=6
https://www.amazon.in/s?k=Books&page=7
https://www.amazon.in/s?k=Books&page=8
https://www.amazon.in/s?k=Books&page=9
https://www.amazon.in/s?k=Books&page=10
https://www.amazon.in/s?k=Books&page=11
https://www.amazon.in/s?k=Books&page=12
https://www.amazon.in/s?k=Books&page=13
https://www.amazon.in/s?k=Books&page=14
https://www.amazon.in/s?k=Books&page=15
https://www.amazon.in/s?k=Books&page=16
https://www.amazon.in/s?k=Books&page=17
https://www.amazon.in/s?k=Books&page=18
https://www.amazon.in/s?k=Books&page=19
https://www.amazon.in/s?k=Books&page=20


In [282]:
main('Books')



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\boraw\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
<ipython-input-281-b37593881384>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [288]:
df = pd.DataFrame(records)

In [289]:
df

,0,1,2,3,4,5
0,"How To Win Friends & Influence People, Interna...",Dale Carnegie,https://amazon.in/gp/slredirect/picassoRedirec...,₹109,4.5 out of 5 stars,"63,852"
1,Great Truths That Set Us Free,unknown,https://amazon.in/gp/slredirect/picassoRedirec...,₹109,4.3 out of 5 stars,121
2,Oswaal CBSE Term 2 Science Class 10 Sample Que...,Oswaal Editorial Board,https://amazon.in/Oswaal-Science-Sample-Questi...,₹111,4.5 out of 5 stars,63
3,The Psychology of Money,Morgan Housel,https://amazon.in/Psychology-Money-Morgan-Hous...,₹321,4.6 out of 5 stars,"30,562"
4,The Power of Your Subconscious Mind,Joseph Murphy,https://amazon.in/Power-Your-Subconscious-Mind...,₹105,4.5 out of 5 stars,"51,029"
5,Atomic Habits: The life-changing million copy ...,James Clear,https://amazon.in/Atomic-Habits-James-Clear/dp...,₹512,4.6 out of 5 stars,"37,396"
6,Rich Dad Poor Dad: What the Rich Teach Their K...,Robert T. Kiyosaki,https://amazon.in/Rich-Dad-Poor-Middle-Updates...,₹372,4.6 out of 5 stars,"56,517"
7,My First Library: Boxset of 10 Board Books for...,Wonder House Books,https://amazon.in/My-First-Library-Boxset-Boar...,₹399,4.5 out of 5 stars,"48,028"
8,My First Book of Pencil Control : Practice Pat...,unknown,https://amazon.in/First-Book-Patterns-Pencil-C...,₹89,4.4 out of 5 stars,"15,915"
9,Brain Activity Book for Kids - 200+ Activities...,unknown,https://amazon.in/Brain-Activity-Book-Kids-Act...,₹95,4.5 out of 5 stars,407


In [290]:
df.shape

(22, 6)